# Identification of target

# Motivation

Plays in which there is a pre-snap motion (motion, shift, or motion at snap) have a higher average expected points added.

The reason for this difference is because on players where there is a pre-snap motion, the completion percentage is generally higher. 

As a defense, it is advantageous to therefore know where the pass is likely headed, based on the look of the offense.

# Solution
- Design a model that predicts the intended receiver on the play
- Then, determine which teams are the most surprising, different etc. 

# Scope
- Pass plays
- Where a player was targeted (ie. no sacks)
- Where time to throw is <= 3.5 seconds (filter out broken plays)

# Train test split
- Weeks 1-6 train
- Weeks 7-9 test

# Architecture 
- XGBoost or LightGBM

In [1]:
import sys
import os
import polars as pl
import pandas as pd
import itables

current_dir = os.getcwd()
sys.path.append(os.path.join(current_dir, "../../", 'src'))

from config import Constants
from preprocessing.preprocessing import BigDataBowlData
from preprocessing.target_prediction_model import TargetPredictionModel
from plotting.plotting import PlotPlayVertical
import seaborn as sns

# init_notebook_mode(all_interactive=True)
pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_rows(100)


GAMES_FILE = "../../data/games.csv"
PLAYS_FILE = "../../data/plays.csv"
PLAYERS_FILE = "../../data/players.csv"
PLAYER_PLAYS_FILE = "../../data/player_play.csv"

# TRACKING_FILES = ["../../data/tracking_week_1.csv",
#                   "../../data/tracking_week_2.csv"
#                   ]

TRACKING_FILES = ["../../data/tracking_week_1.csv",
                  "../../data/tracking_week_2.csv",
                    "../../data/tracking_week_3.csv",
                    "../../data/tracking_week_4.csv",
                    "../../data/tracking_week_5.csv",
                    "../../data/tracking_week_6.csv",
                    "../../data/tracking_week_7.csv",
                    "../../data/tracking_week_8.csv",
                    "../../data/tracking_week_9.csv",
                  ]

data = BigDataBowlData(GAMES_FILE, 
                       PLAYS_FILE, 
                       PLAYERS_FILE, 
                       PLAYER_PLAYS_FILE, 
                       TRACKING_FILES)

In [2]:
target_prediction_model = TargetPredictionModel(data)

In [3]:
filtered_tracking_data = target_prediction_model.process_tracking_df()

In [6]:
filtered_tracking_data["team"].value_counts()

team,count
str,u32
"""offense""",154968
"""defense""",154968
"""football""",14088


In [7]:
frame_data = filtered_tracking_data.filter(pl.col("gameId_playId") == "2022110300_3435").filter(pl.col("frameId") == 66)

In [8]:
frame_data

gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event,week,gameId_playId,wasTargettedReceiver,team
i64,i64,i64,str,i64,str,str,i64,str,str,f64,f64,f64,f64,f64,f64,f64,str,i32,str,i64,str
2022110300,3435,35470,"""Jerry Hughes""",66,"""SNAP""","""2022-11-04 02:59:02.4""",55,"""HOU""","""left""",33.19,24.513333,0.02,0.05,0.02,293.93,336.03,"""ball_snap""",9,"""2022110300_3435""",0,"""defense"""
2022110300,3435,37266,"""Jason Kelce""",66,"""SNAP""","""2022-11-04 02:59:02.4""",62,"""PHI""","""left""",32.24,29.533333,0.13,0.63,0.01,69.08,32.98,"""ball_snap""",9,"""2022110300_3435""",0,"""offense"""
2022110300,3435,39950,"""Lane Johnson""",66,"""SNAP""","""2022-11-04 02:59:02.4""",65,"""PHI""","""left""",31.58,26.663333,0.0,0.0,0.0,100.94,275.04,"""ball_snap""",9,"""2022110300_3435""",0,"""offense"""
2022110300,3435,41300,"""Christian Kirksey""",66,"""SNAP""","""2022-11-04 02:59:02.4""",58,"""HOU""","""left""",36.9,26.953333,0.0,0.0,0.0,306.69,192.65,"""ball_snap""",9,"""2022110300_3435""",0,"""defense"""
2022110300,3435,42441,"""Steven Nelson""",66,"""SNAP""","""2022-11-04 02:59:02.4""",21,"""HOU""","""left""",35.63,37.503333,0.49,2.34,0.04,222.01,166.39,"""ball_snap""",9,"""2022110300_3435""",0,"""defense"""
2022110300,3435,43368,"""Isaac Seumalo""",66,"""SNAP""","""2022-11-04 02:59:02.4""",56,"""PHI""","""left""",31.67,28.233333,0.13,0.96,0.01,31.33,95.39,"""ball_snap""",9,"""2022110300_3435""",0,"""offense"""
2022110300,3435,44922,"""Jaleel Johnson""",66,"""SNAP""","""2022-11-04 02:59:02.4""",90,"""HOU""","""left""",33.57,31.853333,0.0,0.0,0.0,236.35,251.99,"""ball_snap""",9,"""2022110300_3435""",0,"""defense"""
2022110300,3435,44964,"""Desmond King""",66,"""SNAP""","""2022-11-04 02:59:02.4""",25,"""HOU""","""left""",37.09,17.693333,0.73,0.85,0.07,297.18,95.06,"""ball_snap""",9,"""2022110300_3435""",0,"""defense"""
2022110300,3435,45102,"""Zach Pascal""",66,"""SNAP""","""2022-11-04 02:59:02.4""",3,"""PHI""","""left""",32.45,9.143333,0.0,0.0,0.0,82.39,353.46,"""ball_snap""",9,"""2022110300_3435""",0,"""offense"""


In [9]:

offense_data = frame_data.filter(pl.col("team") == "offense")
defense_data = frame_data.filter(pl.col("team") == "defense")
ball_data = frame_data.filter(pl.col("team") == "football")

In [10]:
offense_data

gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event,week,gameId_playId,wasTargettedReceiver,team
i64,i64,i64,str,i64,str,str,i64,str,str,f64,f64,f64,f64,f64,f64,f64,str,i32,str,i64,str
2022110300,3435,37266,"""Jason Kelce""",66,"""SNAP""","""2022-11-04 02:59:02.4""",62,"""PHI""","""left""",32.24,29.533333,0.13,0.63,0.01,69.08,32.98,"""ball_snap""",9,"""2022110300_3435""",0,"""offense"""
2022110300,3435,39950,"""Lane Johnson""",66,"""SNAP""","""2022-11-04 02:59:02.4""",65,"""PHI""","""left""",31.58,26.663333,0.0,0.0,0.0,100.94,275.04,"""ball_snap""",9,"""2022110300_3435""",0,"""offense"""
2022110300,3435,43368,"""Isaac Seumalo""",66,"""SNAP""","""2022-11-04 02:59:02.4""",56,"""PHI""","""left""",31.67,28.233333,0.13,0.96,0.01,31.33,95.39,"""ball_snap""",9,"""2022110300_3435""",0,"""offense"""
2022110300,3435,45102,"""Zach Pascal""",66,"""SNAP""","""2022-11-04 02:59:02.4""",3,"""PHI""","""left""",32.45,9.143333,0.0,0.0,0.0,82.39,353.46,"""ball_snap""",9,"""2022110300_3435""",0,"""offense"""
2022110300,3435,46118,"""Dallas Goedert""",66,"""SNAP""","""2022-11-04 02:59:02.4""",88,"""PHI""","""left""",31.63,33.953333,0.17,0.08,0.02,114.21,56.15,"""ball_snap""",9,"""2022110300_3435""",0,"""offense"""
2022110300,3435,46302,"""Jordan Mailata""",66,"""SNAP""","""2022-11-04 02:59:02.4""",68,"""PHI""","""left""",31.52,32.573333,0.03,0.23,0.0,95.84,171.52,"""ball_snap""",9,"""2022110300_3435""",0,"""offense"""
2022110300,3435,47834,"""A.J. Brown""",66,"""SNAP""","""2022-11-04 02:59:02.4""",11,"""PHI""","""left""",29.77,32.713333,0.69,4.64,0.04,110.86,187.83,"""ball_snap""",9,"""2022110300_3435""",1,"""offense"""
2022110300,3435,47836,"""Miles Sanders""",66,"""SNAP""","""2022-11-04 02:59:02.4""",26,"""PHI""","""left""",27.48,27.473333,0.02,0.03,0.01,83.76,88.08,"""ball_snap""",9,"""2022110300_3435""",0,"""offense"""
2022110300,3435,52461,"""Jalen Hurts""",66,"""SNAP""","""2022-11-04 02:59:02.4""",1,"""PHI""","""left""",27.75,29.593333,0.23,0.17,0.02,96.22,105.57,"""ball_snap""",9,"""2022110300_3435""",0,"""offense"""


In [11]:
data.raw_player_plays.collect()

gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,sackYardsAsOffense,hadPassReception,receivingYards,wasTargettedReceiver,yardageGainedAfterTheCatch,fumbles,fumbleLost,fumbleOutOfBounds,assistedTackle,forcedFumbleAsDefense,halfSackYardsAsDefense,passDefensed,quarterbackHit,sackYardsAsDefense,safetyAsDefense,soloTackle,tackleAssist,tackleForALoss,tackleForALossYardage,hadInterception,interceptionYards,fumbleRecoveries,fumbleRecoveryYards,penaltyYards,penaltyNames,wasInitialPassRusher,causedPressure,timeToPressureAsPassRusher,getOffTimeAsPassRusher,inMotionAtBallSnap,shiftSinceLineset,motionSinceLineset,wasRunningRoute,routeRan,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,pff_defensiveCoverageAssignment,pff_primaryDefensiveCoverageMatchupNflId,pff_secondaryDefensiveCoverageMatchupNflId
i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2022090800,56,35472,"""BUF""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""FALSE""","""NA""","""NA""","""47917""","""NA""","""NA""","""0""","""NA""","""NA""","""NA""","""NA"""
2022090800,56,42392,"""BUF""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""FALSE""","""NA""","""NA""","""47917""","""NA""","""NA""","""0""","""NA""","""NA""","""NA""","""NA"""
2022090800,56,42489,"""BUF""",0,0,0,0,0,1,6,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""TRUE""","""1""","""IN""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,56,44875,"""BUF""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""FALSE""","""NA""","""NA""","""43335""","""NA""","""NA""","""0""","""NA""","""NA""","""NA""","""NA"""
2022090800,56,44985,"""BUF""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""FALSE""","""1""","""OUT""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,56,46076,"""BUF""",0,0,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""FALSE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,56,47857,"""BUF""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""FALSE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,56,47879,"""BUF""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""FALSE""","""1""","""IN""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,56,48512,"""BUF""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""FALSE""","""NA""","""NA""","""41239""","""NA""","""NA""","""0""","""NA""","""NA""","""NA""","""NA"""


In [13]:
data.plays_df.collect()["isDropback"].value_counts()

isDropback,count
bool,u32
false,6388
true,9736


In [ ]:
def create_model_features(frame_data):
    offense_data = frame_data.filter(pl.col("team") == "offense")
    defense_data = frame_data.filter(pl.col("team") == "defense")
    ball_data = frame_data.filter(pl.col("team") == "football")

In [11]:
data.plays_df.head().collect()

gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,playNullifiedByPenalty,absoluteYardlineNumber,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,expectedPoints,offenseFormation,receiverAlignment,playClockAtSnap,passResult,passLength,targetX,targetY,playAction,dropbackType,dropbackDistance,passLocationType,timeToThrow,timeInTackleBox,timeToSack,passTippedAtLine,unblockedPressure,qbSpike,qbKneel,qbSneak,rushLocationType,penaltyYards,prePenaltyYardsGained,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore,distanceToEndzone,playType,preSnapPossessionTeamScore,preSnapDefensiveTeamScore,preSnapPossessionTeamWP,preSnapDefensiveTeamWP,possessionTeamWPAdded,defensiveTeamWPAdded,defensiveTeamExpectedPointsAdded,scoreDifference,possessionTeamNumPlayersInMotionAtSnap,possessionTeamNumPlayersShiftSinceLineset,possessionTeamNumPlayersMotionSinceLineset,playNumTargetedReceivers,playHadPlayersInMotionAtSnap,playHadPlayersMotionSinceLineset,playHadPlayersShiftSinceLineset,playHadMotionAndCameSet,playHadPreSnapMotion,playerMotionCameSetWasTargetted,playerInMotionAtSnapWasTargetted,playerPreSnapMotionWasTargetted,playMotionType,yardsAfterCatch,defensiveTeamNumPlayersInMotionAtSnap,defensiveTeamNumPlayersShiftSinceLineset,defensiveTeamNumPlayersMotionSinceLineset
i64,i64,str,i64,i64,i64,str,str,str,i64,str,i64,i64,str,i64,f64,f64,f64,str,str,i64,str,str,str,str,bool,str,str,str,str,str,str,str,str,str,i64,str,str,str,i64,i64,f64,f64,f64,bool,str,str,i64,str,str,i64,i64,str,str,str,str,i64,i64,i64,str,i64,i64,f64,f64,f64,f64,f64,i64,u32,u32,u32,i64,bool,bool,bool,bool,bool,u32,u32,u32,str,i64,u32,u32,u32
2022102302,2655,"""(1:54) (Shotgun) J.Burrow pass…",3,1,10,"""CIN""","""ATL""","""CIN""",21,"""01:54""",35,17,"""N""",31,0.982017,0.017983,0.719313,"""EMPTY""","""3x2""",10,"""C""","""6""","""36.69""","""16.51""",false,"""TRADITIONAL""","""2.40000009536743""","""INSIDE_BOX""","""2.99""","""2.99000000953674""","""NA""","""FALSE""","""FALSE""","""FALSE""",0,"""NA""","""NA""","""NA""",9,9,0.004634,-0.004634,0.702717,true,"""NA""","""NA""",0,"""Cover-3""","""Zone""",2022,7,"""10/23/2022""","""13:00:00""","""CIN""","""ATL""",35,17,79,"""pass""",35,17,0.982017,0.017983,0.004634,-0.004634,-0.702717,18,0,0,0,1,false,false,false,false,false,0,0,0,"""No motion""",3,0,0,0
2022091809,3698,"""(2:13) (Shotgun) J.Burrow pass…",4,1,10,"""CIN""","""DAL""","""CIN""",8,"""02:13""",17,17,"""N""",18,0.424356,0.575644,0.607746,"""EMPTY""","""3x2""",9,"""C""","""4""","""20.83""","""20.49""",false,"""TRADITIONAL""","""1.13999998569489""","""INSIDE_BOX""","""1.836""","""1.83599996566772""","""NA""","""FALSE""","""FALSE""","""FALSE""",0,"""NA""","""NA""","""NA""",4,4,0.002847,-0.002847,-0.240509,true,"""NA""","""NA""",0,"""Quarters""","""Zone""",2022,2,"""9/18/2022""","""16:25:00""","""DAL""","""CIN""",20,17,92,"""pass""",17,17,0.575644,0.424356,-0.002847,0.002847,0.240509,0,0,0,0,1,false,false,false,false,false,0,0,0,"""No motion""",0,0,0,0
2022103004,3146,"""(2:00) (Shotgun) D.Mills pass …",4,3,12,"""HOU""","""TEN""","""HOU""",20,"""02:00""",3,17,"""N""",30,0.006291,0.993709,-0.291485,"""SHOTGUN""","""2x2""",12,"""C""","""-4""","""26.02""","""17.56""",false,"""TRADITIONAL""","""3.20000004768372""","""INSIDE_BOX""","""2.236""","""2.23600006103516""","""NA""","""FALSE""","""FALSE""","""FALSE""",0,"""NA""","""NA""","""NA""",6,6,0.000205,-0.000205,-0.21848,true,"""NA""","""NA""",0,"""Quarters""","""Zone""",2022,8,"""10/30/2022""","""16:05:00""","""HOU""","""TEN""",10,17,80,"""pass""",3,17,0.006291,0.993709,0.000205,-0.000205,0.21848,-14,0,1,0,1,false,false,true,true,true,0,0,0,"""Pre-snap only""",10,0,0,0
2022110610,348